**1. Variables en base de datos original**  

**Descripcion de Variables:**
Se realiza una descripcion de la variables intentando idenficiar el tipo de cada una, cantidad de nulos, estadisticas relevantes.

In [12]:
import pandas as pd
import numpy as np


ExcelFile = pd.ExcelFile('Ask A Manager Salary Survey 2021 (Responses).xlsx')

#Se lee hoja "Form Responses 1"
df=ExcelFile.parse('Form Responses 1')
display(df.info())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27663 entries, 0 to 27662
Data columns (total 18 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Timestamp                                 27663 non-null  datetime64[ns]
 1   How old are you?                          27663 non-null  object        
 2   Industry                                  27594 non-null  object        
 3   Job title                                 27663 non-null  object        
 4   Additional context on job title           7158 non-null   object        
 5   Annual salary                             27663 non-null  float64       
 6   Other monetary comp                       20505 non-null  float64       
 7   Currency                                  27663 non-null  object        
 8   Currency - other                          185 non-null    object        
 9   Additional context on income

None

,Annual salary,Other monetary comp
count,2.766300e+04,2.050500e+04
mean,1.434379e+05,1.830120e+04
std,5.398599e+06,8.393426e+05
min,0.000000e+00,0.000000e+00
25%,5.400000e+04,0.000000e+00
50%,7.514000e+04,2.000000e+03
75%,1.100000e+05,1.000000e+04
max,8.700000e+08,1.200000e+08


Para las variables categorias que no tienen demasiados valores únicos, se imprime el conteo de ocurrencias por cada valor que pueda tener cada una de ellas, esto para tener una idea acerca de la distribución de los datos en estas variables.

In [28]:
from warnings import catch_warnings
def contarElementos(lista, imprimir=True):
    if(imprimir):
        print("-"*64)
    dic_conteoElementos={}
    listaPosiblesValores=[]

    for elemento in lista:    
        if(dic_conteoElementos.get(elemento, 0) == 0):
            dic_conteoElementos[elemento]=lista.count(elemento)
            listaPosiblesValores.append(elemento)
    try:        
      listaPosiblesValores.sort()
    except:
      print("")
    

    for elemento in listaPosiblesValores:
        if(imprimir):
            try:
              print(elemento +': '+str(dic_conteoElementos[elemento]))
            except:
              print("")
    if(imprimir):
        print("-"*64)
    return listaPosiblesValores

colsCategoricasInteres=df.drop(columns=["Currency - other","Industry","Race","State","Job title","Country","City","Timestamp","Additional context on income","Additional context on job title","Annual salary","Other monetary comp"]).columns
for columna in colsCategoricasInteres:
  print(columna) 
  #print(df[columna].unique())
  contarElementos(list(df[columna]))

How old are you?
----------------------------------------------------------------
18-24: 1159
25-34: 12467
35-44: 9805
45-54: 3151
55-64: 981
65 or over: 89
under 18: 11
----------------------------------------------------------------
Currency
----------------------------------------------------------------
AUD/NZD: 496
CAD: 1651
CHF: 37
EUR: 626
GBP: 1571
HKD: 4
JPY: 23
Other: 145
SEK: 37
USD: 23060
ZAR: 13
----------------------------------------------------------------
Overall years of professional experience
----------------------------------------------------------------
1 year or less: 493
11 - 20 years: 9529
2 - 4 years: 2947
21 - 30 years: 3596
31 - 40 years: 857
41 years or more: 119
5-7 years: 4811
8 - 10 years: 5311
----------------------------------------------------------------
Years of experience in field
----------------------------------------------------------------
1 year or less: 1416
11 - 20 years: 6480
2 - 4 years: 6143
21 - 30 years: 1856
31 - 40 years: 376
41 yea

**2. Variables luego de modeladas.**   

Como tal el nombre de las varibles, por simplicidad se mantiene en inglés, asi mismo, las nuevas variables creadas, se crean con nombres en ingles para mantener en el mismo idioma el nombre de todas las variables.

**Limpieza y procesamiento de datos**


Para la limpieza de los datos para este entregable se enfoco en el campo **“Country”** (en cuento al campo **“City”**, aun no se limpia pues se evidencia una complejidad mucho mas elevada y no hay tiempo suficiente para abordarlo), se opto realizarlo en 2 fases, la primera a groso modo prepara la información para la segunda fase que requiere algo mas de orden en el texto a procesar.

A pesar de enfoncarno el lo Paises, se limpio la varible **Currency**, creando una nueva columna llamada **Currency fix**, pues se evidenciaron casos en donde se tenia la moneda informada tanto en **Currency** como en **Currency - other**, siendo válida esta última.

**Fase 1**  
Se realizan el siguiete procesamiento sobre el pais a tratar primero buscando comas (,) en el texto buscando hacer un split y tener en cuenta solo la primera parte que parece ser la mas relevante, reemplazando tildes por vocales sin tilde, retirando caracteres especiales, dejando todos los textos en minuscula manteniendo en mayúscula solo la primera letra y luego  buscando abarcar los casos mas comunes por medio de expresiones regulares.


In [ ]:
def preparaTexto(textoaLimpiar):
    

    textoaLimpiar = re.split(r',', textoaLimpiar)[0]  # Dividiendo por linea.

    #Reemplaza tildes por vocales sin tilde
    textoaLimpiar = unicodedata.normalize("NFKD", textoaLimpiar).encode("ascii","ignore").decode("ascii")

    #Quita caracters especiales como puntos y comillas - ojo borra tildes
    textoaLimpiar = re.sub(r"[^a-zA-Z0-9]","",textoaLimpiar)

    #Covierte el texto a minúsculas dejando en Mayúsculas son las iniciales de cada palabra
    textoaLimpiar= textoaLimpiar.title()

    #Tratamiento UK
    textoaLimpiar=re.sub('(England|Scotland|Britain|Great Britain|Wales|Ireland|Uk)','United Kingdom',textoaLimpiar)
    textoaLimpiar=re.sub('(Global|CA)','Canada',textoaLimpiar)

    return textoaLimpiar

**Fase 2**  
Se utiliza una función que compara palabras con una lista de palabras y encuentra cual es la más cercana o la mas similar, en este caso la lista de palabras es una lista de paises descargado por medio de la libreria **pycountry**,  esta función fue tomada de https://stackoverflow.com/questions/62677434/replacing-wrong-country-name-with-the-correct-one-in-dataset pero adaptada a este caso en concreto añadiendo una funcionalidad por medio de expresiones regulares.

In [ ]:
def get_most_similar(word):
    word=preparaTexto(word)
    wordlist=paisesValidos;
    top_similarity = 0.0
    most_similar_word = word
    for candidate in wordlist:
        similarity = SequenceMatcher(None,word,candidate).ratio()
        if similarity > top_similarity:
            top_similarity = similarity
            most_similar_word = candidate

    most_similar_word=re.sub('(Usa)','United States',most_similar_word)
    most_similar_word=re.sub('(Uk)','United Kingdom',most_similar_word)
    return most_similar_word

**Procesamiento de datos y creación de nuevas variables**

Se descargaron de la pagina del banco de la república https://www.banrep.gov.co/es/estadisticas/monedas-disponibles las tasas de cambio a COP de todas las monedas al 11/02/2022, y se cargaron como **Tasa Cambio COP.csv**, estas tasas se cruzaron con la base, y con la trm se crearon las columnas:  
 **Salary COP:** Salario Anual en pesos Colombianos **Annual salary** * TRM de la respectiva moneda.   
 **Salary COP fix:**Si el **Salary COP** < 1000usd, se asume que esta mal (en miles) y se multiplica x 1000.  
 **Other monetary comp COP:** Producto de **Other monetary comp** x TRM de la respectiva moneda.  
 **Annual salary fix:** Salario en modena original ajustado, si el **Salary COP** < 1000usd, se asume que esta mal (en miles) y el **Annual salary** se multiplica x 1000.   
 **Salary+Comp COP:** Es la suma de **Salary COP** + **Other monetary comp COP**

In [ ]:
import pandas as pd
import manejo_geografia as geo
import numpy as np


ExcelFile = pd.ExcelFile('5. Visualizacion y Storytelling/Ask A Manager Salary Survey 2021 (Responses).xlsx')
tasasCambio = pd.read_csv('5. Visualizacion y Storytelling/Tasa Cambio COP.csv')

#Se lee hoja "Form Responses 1"
df=ExcelFile.parse('Form Responses 1')

#Python al cargar el excel lee 7 columnas demas, aca las quitamos
df=df.drop(columns=['Additional context on job title','Additional context on income','Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'])

print(df.columns)


print(df.describe())

print(df.info())

print(df.loc[3605:3610,['Country','Currency','Currency - other']])

#Tamaño texo en campo "Currency - other"
df['tamano']=df["Currency - other"].str.len()
df['Currency fix']=df.apply(lambda x: x['Currency'] if x["tamano"]!=3 else x['Currency - other'], axis=1)

#Reemplaza Nan's com 0
df.fillna(0, inplace=True)

#Se hace lefj join con los datos de tasas de cambio para obtener la trm (columna valor)
df=pd.merge(df,tasasCambio,how='left',left_on='Currency fix', right_on='iso4217')
                      
df.fillna(1, inplace=True)

#Calculamos el salario en pesos al multiplicar el valor en moneda original por la trm, 
#Hacemos una validacion y reemplazamos datos cuando el salario anual parace ser muy bajo, es decir si salario anual es inferior a 1000 dolares, lo multiplicamos por mil.
df['Salary COP']=df['Annual salary']*df['valor']

#Revisamos el salario en pesos y si es menor a 10 millones lo multiplicamos por 1000
df['Salary COP fix']=df['Salary COP'].apply(lambda x: x if x > 4000000 else x*1000)

df['Other monetary comp COP']= df['Other monetary comp']*df['valor']

df['Annual salary fix']=df.apply(lambda x: x['Annual salary'] if x["Salary COP"] > 4000000 else x['Annual salary']*1000, axis=1)

df['Salary+Comp COP'] = df['Salary COP fix'] + df['Other monetary comp COP']

'''
Aca esta llamando la funcion get_most_similar definida  en el archivo manejo_geografia
La funcion get_most_similar primero aplicara una seria de expresiones regulares para normalizar un poco los textos
y posteriormente comienza a compara cada nombre de pais con una los nombres de una lista de paises válidos, buscando y
seleccionando así el mas similiar. 
'''
df['Country fix']=df['Country'].apply(lambda x: geo.get_most_similar(x))

**3.Describir paso a paso lo que una persona debería hacer para actualizar los datos y aplicar el modelado que han diseñado.**  

-Descargar el archivo **Ask A Manager Salary Survey 2021 (Responses).xlsx** de https://docs.google.com/spreadsheets/d/1IPS5dBSGtwYVbjsfbaMCYIWnOuRmJcbequohNxCyGVw/edit?resourcekey#gid=1625408792

-Descargar de la página del banco de la república las tasas de cambio a peso Colombiano, filtrar el archivo por tipo_cambio="Pesos colombianos por cada moneda" y tipo_tasa="media", una vez hecho esto crear el archivo **Tasa Cambio COP.csv**  

-Crear el archivo manejo manejo_geografia.py con las funciones def preparaTexto(textoaLimpiar) y get_most_similar(word) previamente vistas, incluir el listado de paises válidos en este caso se uso libreria pycountry, pero podria ser otra fuente.

-Correr el codigo python que carga el excel  Ask A Manager Salary Survey 2021 (Responses).xlsx y el archivo Tasa Cambio COP.csv visto en el paso anterior.

-Una vez ejecutado exportar la base limpia, ya procesada y con las nuevas columnas y subirla a Google Data Studio.